# <font color = "#018786"> Network analysis </font>

We will perform a network analysis on the network of  comic book characters from the Marvel Universe. We decided to focus on the first 10.000 characters, sorted from A to Z. For each character we collected their wikia page and which nationality they belong to (for community detection). This data was accessible by parsing the relevant web pages at http://marvel.wikia.com/wiki. Below, we will describe the different steps in our network analysis:

* 0\. Load Modules  
* 1\. Data Extraction
    * 1.1\. Extract Characters
    * 1.2\. Extract Nationalities
    * 1.3\. Extract Links
    * 1.4\. Data Merging
    * 1.5\. Data Cleaning
* 2\. Network Analysis
    * 2.1\. Communities
    * 2.2\. Network Visualization
    * 2.3\. Degree Distribution
    * 2.4\. Modularity
    * 2.5\. Confusion Matrix 

## 0.  **Load Modules**

In [3]:
import json, operator, community #JSON parser, efficient operator functions, python-louvain package
from urllib2 import urlopen #extensible library for opening URLs
import urllib # package for working with URL's
from bs4 import BeautifulSoup #pulling data out from HTML and XML files
from prettytable import PrettyTable #displaying tabular data in a ASCII table format
import matplotlib.pyplot as plt # plotting figures
import numpy as np # powerful calculation library
import networkx as nx # networks creation library
from networkx.drawing.nx_agraph import graphviz_layout
import cPickle as pickle # python data structures as files
%matplotlib inline
from fa2 import ForceAtlas2
from itertools import count

In [4]:
def prettifyName(name):
    name = name.replace('_',' ') if '_' in name else name
    return name

## 1.  **Data Extraction**
### 1.1 Extract Characters
With *getCharacters()* function, we retrieve 10 000 characters from the Marvel wikia page. For every character, we save his ID, nationality and set of all of their links in a dictionary.

In [6]:
def getCharacters():

    response = urlopen('http://marvel.wikia.com/api/v1/Articles/List?expand=1&category=Characters&limit=10000')
    html = response.read()
    j = json.loads(html)
    characters = dict()
    c = j.get("items")
    
    for a in c:
        name = (a.get("title")).replace(' ', '_')
        idName = (a.get("id"))
        characters[name] = {'id': idName, 'page' : set(), 'nationality' : "Alien"}
    
    return characters

In [7]:
characters = getCharacters()

In [8]:
len(characters)

10000

### 1.2 Extract Nationalities
With *getNationalities()* function, we retrieve the nationalities from the Marvel wikia page. Save them in a list.

In [9]:
 def getNationalities():
    response = urlopen('http://marvel.wikia.com/api/v1/Articles/List?expand=1&category=Characters_by_Nationality&limit=10000')
    html = response.read()
    j = json.loads(html)
    c = j.get("items")

    nationalities = []
    for a in c:
        nationality = (a.get("title")).replace(' ', '_')
        nationalities.append(nationality)
    
    return nationalities

In [10]:
nationalities = getNationalities()

In [11]:
len(nationalities)

333

### 1.3 Data Merging
With *getNationalitiesForCharacters()* function, we connect the origin information with the characters and update the dictionary with the new information.

In [12]:
def getNationalitiesForCharacters(characters, nationalities):
    for nationality in nationalities:
        try:
            #print("nationality is " + nationality)
            response = urlopen(r'http://marvel.wikia.com/api/v1/Articles/List?expand=1&category=%s&limit=10000' % nationality)
            html = response.read()
            chars = json.loads(html)
            c = chars.get("items")
            for a in c:
                try:
                    name = (a.get("title")).replace(' ', '_')
                    characters[name]['nationality'] = nationality
                except KeyError:
                    pass
        except:
            print("nationality got an http error" + nationality)      
    return characters

In [13]:
getNationalitiesForCharacters(characters, nationalities)

nationality got an http errorBagmoman
nationality got an http errorBelizean
nationality got an http errorCommorians
nationality got an http errorKeshanis
nationality got an http errorLatkovians
nationality got an http errorMacedonian
nationality got an http errorMeroë_citizens
nationality got an http errorMonacoan
nationality got an http errorNone
nationality got an http errorNor-Am_Pact_region
nationality got an http errorPuntians
nationality got an http errorRudyardians
nationality got an http errorSudanese
nationality got an http errorYamatai
nationality got an http errorZamboulan
nationality got an http errorZenoshans


{u'Alyssa_Moy_(Earth-616)': {'id': 19623,
  'nationality': 'Alien',
  'page': set()},
 u'Cessily_Kincaid_(Earth-600123)': {'id': 501558,
  'nationality': u'Americans',
  'page': set()},
 u'Andrea_Vecchiato_(Earth-1610)': {'id': 780294,
  'nationality': u'Italians',
  'page': set()},
 u'Adelynn_Duquesne_(Earth-616)': {'id': 738533,
  'nationality': u'French',
  'page': set()},
 u'Alexi_Alexivitch_(Earth-616)': {'id': 1113731,
  'nationality': u'Soviets',
  'page': set()},
 u'Beltan_(Earth-616)': {'id': 1067447,
  'nationality': u'Cimmerians',
  'page': set()},
 u'Belle_(A.I.)_(Earth-TRN659)': {'id': 1108390,
  'nationality': 'Alien',
  'page': set()},
 u'Abner_Jenkins_(Earth-TRN005)': {'id': 435584,
  'nationality': u'Latverians',
  'page': set()},
 u'Alexander_Flynn_(Earth-616)': {'id': 30106,
  'nationality': u'Latverians',
  'page': set()},
 u'Adele_Santiago_(Earth-616)': {'id': 872313,
  'nationality': u'Americans',
  'page': set()},
 u'Black_Widow_(Earth-61211)': {'id': 773233,
  '

With *getNations()* function, we retrieve all nationalities in the Marvel universe, with *minCount* members.

In [14]:
def getNations(minCount=0):
    nations = dict()
    for k,v in characters.iteritems():
        n = v['nationality']
        if n != None:
            if n in nations:
                nations[n]['count'] += 1
                nations[n]['members'].append(k)
            else:
                nations[n] = dict()
                nations[n]['count'] = 1
                nations[n]['members'] = list()
                nations[n]['members'].append(k)
    nations = {k: v for k,v in nations.iteritems() if v['count'] > minCount}
    return nations

In [15]:
nations = getNations()

In [16]:
len(nations)

187

### 1.4 Extract Links
With *getLinks()* function, we retrieve all links from the web page of a character, and update the characters dictionary with the new information.

In [17]:
def getLinks(characters):
    baseUrl = 'http://marvel.wikia.com/wiki/'
    for char,v in characters.iteritems():
        url = baseUrl + char
        try:
            response = urlopen(url.encode("utf-8"))
            source = response.read()
    
            soup = BeautifulSoup(source)
            div = soup.find(id="mw-content-text")
            ps = div.findChildren('p')
            for p in ps:
                hr = p.findChildren('a')
                if p.parent.name != 'td':
                    for h in hr:
                        if h.has_attr('href'):
                            if h['href'].startswith('/wiki'):
                                ele = h['href'].split('/')[-1:][0]
                                if '?' not in ele and 'Category' not in ele and not ele.startswith('File'):
                                    ele = ele.decode('unicode_escape').encode('ascii','ignore')
                                    v['page'].add(u''+ele) 
        except:
            pass
    return characters

In [ ]:
characters = getLinks(characters)

In [ ]:
#check how many empty sets we have #This is just testing
emptySets = 0
for key, value in characters.items():
    if len(value['page']) == 0:
        emptySets =emptySets + 1
print emptySets

### 1.5 Data Cleaning
We remove all characters with special symbols.

In [ ]:
#Gives an error when we restart this function, without cleaning the characters dict          
for v in characters.values():
    for l in v['page']:
        if "%27" or "%C3%" in l:
            v['page'].remove(l)
            n = urllib.unquote(l)
            v['page'].add(n)
        elif "\xc3" or "\xe8" in l:
            l.encode("utf-8")

## 2.  **Data Extraction**
### 2.1 Communities
With *printNationsInfo()* function, we print how many communities have been found and how many charachers do they include.

In [ ]:
def printNationsInfo(nations):
    sorted_nations = sorted(nations.items(), key=operator.itemgetter(1), reverse=True)
    nation_table = PrettyTable(['Nationality', 'Number of characters'])
    for n in sorted_nations:
        nation_table.add_row([n[0], n[1]['count']])
    return nation_table

In [ ]:
print printNationsInfo(nations)

### 2.2 Network Visualization

 - Create a directed graph over the characters and their links.

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(characters.keys())

In [ ]:
len(G.nodes)

In [ ]:
for k,v in characters.iteritems():
    for name in characters.keys():
        if (name in v['page'] and name !=k):
            G.add_edge(k, name)

In [ ]:
len(G.edges)

 - Remove all of the isolated Nodes

In [ ]:
isolatedNodes = nx.isolates(G)
G.remove_nodes_from(list(isolatedNodes))

In [ ]:
len(G.nodes)

In [ ]:
plt.figure(1,figsize=(12,12))
pos = nx.spring_layout(G)
nx.draw(G, node_size=30, node_color="#FF0000", node_shape='o', edge_color='.25', with_labels=False, width=.5, pos=pos)
plt.title('Full Marvel network')
plt.show()

 - Convert to undirected graph

In [ ]:
U = G.to_undirected()

In [ ]:
forceatlas2 = ForceAtlas2(
                          # Behavior alternatives
                          outboundAttractionDistribution=False,  # Dissuade hubs
                          edgeWeightInfluence=0.4,

                          # Performance
                          jitterTolerance=0.2,  # Tolerance
                          barnesHutOptimize=True,
                          barnesHutTheta=0.2,

                          # Tuning
                          scalingRatio=1.0,
                          strongGravityMode=False,
                          gravity=0.1,

                          # Log
                          verbose=True
)
    
positions = forceatlas2.forceatlas2_networkx_layout(U,pos=None, iterations=1000)
#enlarge plot
plt.figure(1, figsize=(12,12))
nx.draw(U, positions, width = 0.2, node_size=5, with_labels=False)
plt.title('Full Marvel network')
plt.axis('off')
plt.show()

In [ ]:
G_wcc = max(nx.weakly_connected_component_subgraphs(G),key=len)

In [ ]:
plt.figure(2,figsize=(12,12))
pos = nx.layout.fruchterman_reingold_layout(G_wcc)
nx.draw(G_wcc, node_size=30, node_color="#FF0000", node_shape='o', edge_color='.25', with_labels=False, width=.5, pos=pos)
plt.title('The largest component in the Marvel network')
plt.show()

### 2.3 Degree Distribution
 - With *printGraphStats()* we print some statistics of the graph such as degree, betweenness centrality and eigenvector centrality.

In [ ]:
def printGraphStats(G_wcc, top=10):
    sorted_G_out_deg = sorted(G_wcc.out_degree(), key=operator.itemgetter(1), reverse=True)
    sorted_G_in_deg = sorted(G_wcc.in_degree(), key=operator.itemgetter(1), reverse=True)
    deg_table = PrettyTable(['Character - In degree',' Character - Out degree'])
    
    for i in range(top):
        a = ["%s, %d" % (prettifyName(sorted_G_in_deg[i][0]), sorted_G_in_deg[i][1]),
            "%s, %d" % (prettifyName(sorted_G_out_deg[i][0]), sorted_G_out_deg[i][1])]
        deg_table.add_row(a)
    
    G_bc = nx.betweenness_centrality(G_wcc)
    G_bc_sorted = [(k, G_bc[k]) for k in sorted(G_bc, key=G_bc.get, reverse=True)]

    bc_table = PrettyTable(['Character', 'Betweenness centrality'])
   
    for i in range(top):
        a = [prettifyName(G_bc_sorted[i][0]),
            "%.3f" % G_bc_sorted[i][1]]
        bc_table.add_row(a)

    ec_table = PrettyTable(['Character', 'Eigenvector centrality (In-degree)', 'Eigenvector centrality (Out-degree)'])
    G_ec_in = nx.eigenvector_centrality(G_wcc)
    G_ec_out = nx.eigenvector_centrality(G_wcc.reverse())
    G_ec_in_sorted = [(k, G_ec_in[k]) for k in sorted(G_ec_in, key=G_ec_in.get, reverse=True)]
    G_ec_out_sorted = [(k, G_ec_out[k]) for k in sorted(G_ec_out, key=G_ec_out.get, reverse=True)]    
    
    
    for i in range(top):
        a = [prettifyName(G_ec_in_sorted[i][0]), 
             "%.4f" % G_ec_in_sorted[i][1],
            "%.4f" % G_ec_out_sorted[i][1]
            ]
        ec_table.add_row(a)
    return deg_table, bc_table, ec_table

 - We display Top 5 charachers with highest **In Degree and Out Degree**

* **In Degree**
 * Anthony Stark - Iron man
 * Bruce Banner - Hulk
 * Benjamin Grimm - The Thing from Fantastic 4
 * Charles Xavier - Professor X
 * Carol Danvers - Captain Marvel

* **Out Degree**
 * Bruce Banner - Hulk
 * Charles Xavier - Professor X
 * Anthony Stark - Iron man
 * Alison Blaire - Dazzler
 * Alexander Summers - Havok

In [ ]:
top = 5
deg_table, bc_table, ec_table = printGraphStats(G_wcc, top=top)
print "The top %d Marvel characters with highest In and Out degree" %top
print deg_table

- We display Top 5 charachers with highest **Betweenness Centrality**
 * Bruce Banner - Hulk
 * Anthony Stark - Iron man
 * Charles Xavier - Professor X
 * Alison Blaire - Dazzler
 * Ares - Ares, God of war

In [ ]:
print "\nThe top %d Marvel characters with highest Betweenness Centrality:" % top   
print bc_table

- We display Top 5 charachers with highest **Eigenvector centrality (In and Out degree)**
 * Bruce Banner - Hulk
 * Charles Xavier - Professor X
 * Carol Danvers - Captain Marvel
 * Anthony Stark - Iron man
 * Alison Blaire - Dazzler

In [ ]:
print "\nThe top %d Marvel characters with highest Eigenvector centrality (In and Out degree)" % top
print ec_table

### 2.4 Modularity
$$M=\sum\limits_{c=1}^{n_c}\left[\frac{L_c}{L}-\left(\frac{k_c}{2L}\right)^2 \right]$$

$n_c$: Number of communities  
$L_c$: The total number of links within the community $C_c$  
$k_c$: The total degree of the nodes in the community $C_c$  
A node's *total degree*, $k_i$ is given by: $k_i=k_i^{in}+k_i^{out}$

In [ ]:
def calculateModularity(nations):
    n_c = len(nations)
    m1 = 0; L1 = 0; k1 = 0; L = 0
    for nation, v in nations.iteritems():
        v['L'] = int(); v['k'] = int(); v['M'] = float(0);
        for member in v['members']:
            if type(G.degree(member)) is int:
                v['k'] = v['k'] + (G.degree(member))
            for m in G.edges(member):
                if (m[0] != member and m[0] in v['members']) or (m[1] != member and m[1] in v['members']):
                    v['L'] = v['L'] + 1
    for v in nations.values():
        L = L+ v['k']
    for k,v in nations.iteritems():
        l_c = float(v['L']); k_c = float(v['k'])
        v['M'] = (l_c / L) - (pow((k_c / (2 * L)), 2.0))
    return nations

In [ ]:
nation = calculateModularity(getNations(4))

In [ ]:
def nationModularityTable(nations):
    mt = PrettyTable(['Nation', 'L_c', 'k_c', 'M_c'])
    for k, v in nations.items():
        mt.add_row([k, v['L'], v['k'], "%.5f" % v['M']])
    return mt

In [ ]:
nation = calculateModularity(getNations(10))
print nationModularityTable(nation),'\n'

 - Retrieve the communities using the Louvain algorithm, and print the corresponding network.

In [ ]:
plt.figure(figsize=(10,10))
partition = community.best_partition(U)
size = float(len(set(partition.values())))
pos = nx.spring_layout(U)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    nx.draw_networkx_nodes(U, pos, list_nodes, node_size = 20, node_color = str(count / size))
nx.draw_networkx_edges(U, pos, alpha=0.5)
plt.axis('off')
plt.show()
 

 - Calculate the confusion matrix.

In [ ]:
def calculateConfusionMatrix(U):
    partition = community.best_partition(U)
    nations = getNations(30)
    modularity = community.modularity(partition, U)
    noCommunities = max(partition.values())
    D = np.zeros((len(nations), noCommunities))
    for i in range(len(nations)):
        for j in range(noCommunities):
            for v in partition:
                if partition[v] == j:
                    if v in nations.values()[i]['members']:
                        D[i,j] += 1
    return D, modularity, noCommunities

In [ ]:
D, modularity, noCom = calculateConfusionMatrix(U)

In [ ]:
print "Confusion matrix:\n", D

 - Calculate the distribution of the diagonal elements

In [ ]:
def confusionMatrixDist(D):
    diag = [D[i,j] for i in range(D.shape[0]) for j in range(D.shape[1]) if i == j]
    a, b = np.histogram(diag)

    plt.figure(1)
    plt.plot(b[:-1], a)
    plt.yticks(range(int(max(a))+1))
    plt.title("Distribution of the diagonal elements")
    plt.show()

In [ ]:
confusionMatrixDist(D)